# Solving Coin Flip Game using Monte Carlo

## Game Description

A player has three turns.  During each turn they can flip a coin or pass.  They accumulate money in their pot during each turn.  At the end of three turns, they get to keep the money in the pot.  

- If they "pass" during a turn, then the game ends and they keep the money in the pot.   
- If they "flip" during a turn, then a random 50/50 coin is flipped.  If it lands heads, then 1 dollar is added to the pot.  If it lands tails, then all the money is removed from the pot.  


Our goal is to figure out how to flip/pass at each turn to maximize our average winnings.  

## State
The time steps are $k = \{0,1,2,3\}$.   During each time step $x_k$ is the amount of money in the pot which can range from 0 to 3.    We let state $s$ be a function

$
s = k*4 + x_k
$

Thus there are 16 states even though some of them are unrealizable in the game (for instance state $s=2$ is $k=0$ and $x_0 = 2$ which is impossible since we can't start the game with 2 dollars in the pot).  
## Equations

We let $v[(k,x)]$ represent the value of being in state $x$ at turn $k$; that is, with $x$ dollars in the pot at turn $k$.   The value $v[0,0]$ is the expected value we can earn in this game starting at the start state with 0 dollars in the pot.  That is the ultimate value we seek.

Let $Q[(k,x),a]$ be the action-value function.  The state we are in is state $(k,x)$ and the action is $a$.   This is the expected value of starting in this state and taking this action.   

Let $P[(k,x)]$ be the policy -- the best action choice at state $(k,x)$.   We let 0=pass and 1=flip.  




In [1]:
import numpy as np

## Key Monte Carlo and System Functions

In [2]:

#------------------------------------------------------------- check
def update (s,a):
  '''
  Compute next state given state s and action a
  s = current state
    s%4 = number of dollars in pot
    s//4 = turn number k=0,1,2,3
  a = 0 means pass
  a = 1 means flip
  return state, reward (0 for all trajectories)
  '''
  turn = s // 4
  pot = s % 4

  if a == 0:
    return pot+12, 0

  #flip = np.random.randint(1,3)
  #if flip == 1:
  if np.random.random() < 0.5:
    return (turn+1)*4,0      # tails
  else:
    return pot+1+(turn+1)*4,0    # heads

#------------------------------------------------------------- check
def e_greedy(s,policy,epsilon):
  '''
  Implements an e-greedy policy.
  With probability epsilon, it returns random action choice
  otherwise returns action choice specified by the policy

  s = current state
  policy = policy function (an array that is indexed by state)
  epsilon (0 to 1) a probability of picking exploratory random action
  '''
  r = np.random.random()
  if r > epsilon:
    return policy[s]
  else:
    return np.random.randint(0,2)

#------------------------------------------------------------- check
def make_trajectory (policy,epsilon):
  '''
  Simulate one trajectory of experience
  Return list of tuples during trajectory
  Each tuple is (s,a,r) -> state / action / reward
  epsilon = probability of exploratory action
  '''
  traj = []
  k=0
  s=0   #  turn 0, pot 0

  # s < 12 implies turn k < 3
  while (s < 12):
    a = e_greedy(s,policy,epsilon)
    s_prev = s
    s,r = update(s,a)
    traj.append((s_prev,a,r))

  # final reward = state value, final action = 0 (meaningless)
  traj.append((s,0,s%4))
  return traj

#------------------------------------------------------------- check
def init ():
  '''
  Create totals, counts and policy defaults
  '''
  totals = np.zeros((16,2))
  counts = np.zeros((16,2)).astype(int)
  P = np.zeros(16).astype(int)
  return totals,counts,P

#-------------------------------------------------------------
def policy_improvement(Q):
  '''
  Update value function V and policy P based on Q values
  '''
  V = np.max(Q,axis=1)
  P = np.argmax(Q,axis=1)
  return V,P

#-------------------------------------------------------------
def policy_evaluation(totals,counts,policy,n,epsilon):
  '''
  do n trajectories of learning
  and update the v/count arrays
  '''
  for i in range(n):
    t = make_trajectory(policy,epsilon)
    m = len(t)
    #print(m,t)
    sum_r = np.zeros(m)
    sum_r[m-1] = t[-1][2]
    for j in range(m-2,-1,-1):
      sum_r[j] = sum_r[j+1] + t[j][2]
    #print(sum_r)

    for j in range(m):
      s,a,r = t[j]
      counts[s,a] += 1
      totals[s,a] += sum_r[j]

#-------------------------------------------------------------
def policy_iteration(totals,counts,policy,Q,n,m,epsilon):
  '''
  Perform n iterations of policy iteration
  using m trials (episodes) per policy update
  '''
  for i in range(n):
    Q = compute_Q(totals,counts)
    V,P = policy_improvement(Q)
    policy_evaluation(totals,counts,P,m,epsilon)

  print(counts)
  Q = compute_Q(totals,counts)
  V,P = policy_improvement(Q)


#-------------------------------------------------------------
def compute_Q (totals,counts):
  '''
  Compute the Q values based on totals and counts (average)
  '''
  Q = np.zeros((16,2))
  for i in range(len(totals)):
    for a in range(2):
      if counts[i][a] > 0:
        Q[i][a] = totals[i][a] / counts[i][a]
      else:
        Q[i][a] = 0
  return Q


#-------------------------------------------------------------
def assess (policy,trials):
  '''
  Assess the value of the current policy by completing #trials
  using the specified policy (no e-greedy random actions)
  Does not accrue learning experience nor change policy
  '''
  totals,counts,P = init()
  policy_evaluation(totals,counts,policy,trials,0)
  Q = compute_Q(totals,counts)
  V,P = policy_improvement(Q)
  return V[0]

### Test Trajectory
Run a couple of trajectories and test their accuracy

In [3]:
totals,counts,P = init()
#np.random.seed(seed=42)
epsilon = 0.3
print('P =',P)
sumt = 0
k = 10
for i in range(k):
  t = make_trajectory(P,epsilon)
  sumt += t[-1][-1]
  if k <= 10:
    print(t)

print("Average Game Value:",sumt/k)

P = [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]


[(0, np.int64(0), 0), (12, 0, 0)]
[(0, 1, 0), (4, np.int64(0), 0), (12, 0, 0)]
[(0, np.int64(0), 0), (12, 0, 0)]
[(0, 0, 0), (12, 0, 0)]
[(0, 1, 0), (4, np.int64(0), 0), (12, 0, 0)]
[(0, 0, 0), (12, 0, 0)]
[(0, np.int64(0), 0), (12, 0, 0)]
[(0, 0, 0), (12, 0, 0)]
[(0, np.int64(0), 0), (12, 0, 0)]
[(0, np.int64(0), 0), (12, 0, 0)]
Average Game Value: 0.0


### Do MC Learning
This next block does a real segment of MC learning
- Start with initial (blank) learning experience
- Do 1000 iterations of Policy Iteration each with 100 trials of experience
- Extract policy, value function and Q values

In [4]:
totals,counts,P = init()
m = 100
n = 5000
epsilon = 0.1
Q = compute_Q(totals,counts)
V,P = policy_improvement(Q)
policy_iteration(totals,counts,P,Q,n,m,epsilon)

Q = compute_Q(totals,counts)
V,P = policy_improvement(Q)
print('Q =\n',Q)
print('V =\n',V)
print('P =\n',P)

[[ 25229 474771]
 [     0      0]
 [     0      0]
 [     0      0]
 [ 11924 224742]
 [ 14019 224086]
 [     0      0]
 [     0      0]
 [ 11175 213506]
 [106519   5812]
 [106289   5527]
 [     0      0]
 [160502      0]
 [227556      0]
 [109158      0]
 [  2784      0]]
Q =
 [[0.         0.95672229]
 [0.         0.        ]
 [0.         0.        ]
 [0.         0.        ]
 [0.         0.73788611]
 [1.         1.22440045]
 [0.         0.        ]
 [0.         0.        ]
 [0.         0.50124118]
 [1.         0.98726772]
 [2.         1.51112719]
 [0.         0.        ]
 [0.         0.        ]
 [1.         0.        ]
 [2.         0.        ]
 [3.         0.        ]]
V =
 [0.95672229 0.         0.         0.         0.73788611 1.22440045
 0.         0.         0.50124118 1.         2.         0.
 0.         1.         2.         3.        ]
P =
 [1 0 0 0 1 1 0 0 1 0 0 0 0 0 0 0]


### Pretty Output

In [5]:
for k in range(4):
  for pot in range(k+1):
    s = k*4 + pot
    print("Turn [{0:d}]  Pot {3:d}   |  Policy: {1:4s}   Value: {2:4.2f}".format(k,"stay" if P[s]==0 else "flip",V[s],pot))
  print()

Turn [0]  Pot 0   |  Policy: flip   Value: 0.96

Turn [1]  Pot 0   |  Policy: flip   Value: 0.74
Turn [1]  Pot 1   |  Policy: flip   Value: 1.22

Turn [2]  Pot 0   |  Policy: flip   Value: 0.50
Turn [2]  Pot 1   |  Policy: stay   Value: 1.00
Turn [2]  Pot 2   |  Policy: stay   Value: 2.00

Turn [3]  Pot 0   |  Policy: stay   Value: 0.00
Turn [3]  Pot 1   |  Policy: stay   Value: 1.00
Turn [3]  Pot 2   |  Policy: stay   Value: 2.00
Turn [3]  Pot 3   |  Policy: stay   Value: 3.00



### Assessment
We can also assess the current policy by conduction many non-learning trials.

In [6]:
value = assess(P,2000)
print(value)

1.0025
